In [1]:
import mysql.connector
from path import *
from mysql.connector import Error as MySqlError
from bson.json_util import loads
from constants import *
import os

### Connection 

In [3]:
PW = 'root'

def create_db_connection(host_name, user_name, user_password, db_name):
	connection = mysql.connector.connect(
		host=host_name,
		user=user_name,
		passwd=user_password,
		database=db_name
	)
	return connection

CONNECTION = create_db_connection("localhost", "root", PW, 'maadb_project')

### Utils

In [4]:
def retrieve_data_from_json(path):
	with open(path, 'r', encoding='utf8') as f:
		return loads(f.read())

def get_data_from_db(query):
    cursor = CONNECTION.cursor()
    cursor.execute(query)
    return cursor

def exec_query(query):
	cursor = CONNECTION.cursor()
	cursor.execute(query)
	CONNECTION.commit()
	

In [5]:
sentiments = ['anger', 'anticipation', 'disgust', 'fear', 'hate', 'hope', 'joy', 'like', 'love', 'sadness', 'surprise', 'trust', 'pos', 'neg']

resources = {
	'anger': ['EmoSN', 'NRC', 'sentisense'],
	'anticipation': ['NRC', 'sentisense'],
	'disgust': ['NRC', 'sentisense'],
	'hate': ['sentisense'],
	'fear': ['NRC', 'sentisense'],
	'hope': ['sentisense'],
	'joy': ['EmoSN', 'NRC', 'sentisense'],
	'like': ['sentisense'],
	'love': ['sentisense'],
	'neg': ['GI', 'HL', 'ET', 'LIWC'],
	'pos': ['GI', 'HL', 'ET', 'LIWC'],
	'sadness': ['NRC', 'sentisense'],
	'surprise': ['NRC', 'sentisense'],
	'trust': ['NRC']
}

#### Creazione tabella sentiment

In [6]:
#exec_query("drop table maadb_project.sentiment", connection)
query = """CREATE TABLE `maadb_project`.`sentiment` (
			`nome` VARCHAR(20) NOT NULL, 
			PRIMARY KEY (`nome`));"""
exec_query(query)

#### Inserimento dati tabella sentiment

In [7]:
def insert_sentiments():
	query = """INSERT INTO `maadb_project`.`sentiment` (`nome`) VALUES"""
	insert = """('{}'),"""
	for s in sentiments:
		query += insert.format(s)
	query = query[:-1] + ";"
	exec_query(query)

insert_sentiments()

#### Creazione tabella lexical_resource

In [8]:
#exec_query("drop table maadb_project.lexical_resource", connection)
query = """CREATE TABLE `maadb_project`.`lexical_resource` (
			`id_lex_res` INT NOT NULL AUTO_INCREMENT,
			`sentiment` VARCHAR(20) NOT NULL,
			`resource_name` VARCHAR(10) NOT NULL,
			`number_of_words` INT NOT NULL,
			PRIMARY KEY (`id_lex_res`),
			CONSTRAINT `sentiment_lexres`
			FOREIGN KEY (`sentiment`) REFERENCES `maadb_project`.`sentiment` (`nome`)
			ON DELETE RESTRICT ON UPDATE CASCADE);
		"""
exec_query(query)

#### Inserimento dati tabella lexical_resources

In [9]:
def insert_lexical_resources():
	path = f'{PREP_PATH}/LexResources.json'
	lex_res = retrieve_data_from_json(path)
	query = """INSERT INTO `maadb_project`.`lexical_resource` (`sentiment`, `resource_name`, `number_of_words`) VALUES"""
	insert = """('{}', '{}', '{}'),"""
	for item in lex_res:
		query += insert.format(item["sentiment"], item["id"], item["totNumberWords"])
	query = query[:-1] + ";"
	exec_query(query)
	print("Lexical Resources correctly inserted")

insert_lexical_resources()

Lexical Resources correctly inserted


#### Creazione tabella word

In [10]:
#exec_query("drop table maadb_project.word")
query = """CREATE TABLE `maadb_project`.`word` (
			`id_word` INT NOT NULL AUTO_INCREMENT,
			`lemma` VARCHAR(200) NOT NULL,
			PRIMARY KEY (`id_word`));"""
exec_query(query)

#### Inserimento dati tabella word

In [12]:
#circa 1.20 min
def insert_word_from_lexical_resource(path):
    data = retrieve_data_from_json(path)
    query = "INSERT INTO word (lemma) VALUES"
    insert = '("{}"),'
    for res in data:
        lemma = res["lemma"]
        query += insert.format(lemma)
    query = query[:-1] + ";"
    exec_query(query)
    print("Inserimento parole risorse lessicali completato!")

def insert_word_from_tweet(path):
    data = retrieve_data_from_json(path)
    query = "INSERT INTO word (lemma) VALUES"
    insert = '("{}"),'
    for tweet in data:
        for word in tweet["words"]:
            lemma = word["lemma"]
            query += insert.format(lemma)
    query = query[:-1] + ";"
    exec_query(query)

def insert_word_on_db():
    path = os.path.join(PREP_PATH, "LexResourcesWords.json")
    insert_word_from_lexical_resource(path)
    for sent in sentiments:
        path = os.path.join(PREP_PATH, f"preprocessed_tweets_{sent}.json")
        if os.path.exists(path):
            insert_word_from_tweet(path)
        print(f"Completed {sent}")
    print("Word correctly inserted")

insert_word_on_db()

Inserimento parole risorse lessicali completato!
Completed anger
Completed anticipation
Completed disgust
Completed fear
Completed hate
Completed hope
Completed joy
Completed like
Completed love
Completed sadness
Completed surprise
Completed trust
Completed pos
Completed neg
Word correctly inserted


#### Creazione tabella token

In [13]:
#exec_query("drop table maadb_project.token")
query = """CREATE TABLE `maadb_project`.`token` (
			`id_token` INT NOT NULL AUTO_INCREMENT,
			`token` VARCHAR(100) NOT NULL,
			`type` VARCHAR(20) NOT NULL,
			PRIMARY KEY (`id_token`));"""
exec_query(query)

#### Inserimento dati tabella token

In [ ]:
#Il problema è questa emoticon :\ per vari motivi:
#1. dal sistema può essere letta solo se la scrivi :\\ in modo che si indichi che si vuole effettivamente il back slash
#2. una volta che riconosco quel tokene provo a passarlo nel formato esso viene rappresentato come la sua effettiva emoticon, ossia :\
#3. questa rappresentazione non va bene nella query e manda in errore

def insert_token_of_sent_on_db(path):
    data = retrieve_data_from_json(path)
    query = """INSERT INTO token (token, type) VALUES"""
    insert = '\n("{}", "{}"),'
    insert_2 = """\n('{}', '{}'),"""
    i = 0
    for tweet in data:
        for type in ['emojis', 'emoticons']:
            for token in tweet[type]:
                if "'" in token:
                    query += insert.format(token, type)
                else:
                    query += insert_2.format(token, type)
        i += 1
    query = query[:-1] + ";"

    exec_query(query)

def insert_token_on_db():
    for sent in sentiments:
        print(sent)
        path = os.path.join(PREP_PATH, f"preprocessed_tweets_{sent}.json")
        if os.path.exists(path):
            insert_token_of_sent_on_db(path)
    print("Non-word tokens correctly inserted")

insert_token_on_db()

#### Creazione tabella tweet

In [14]:
#exec_query("drop table maadb_project.tweet")
query = """CREATE TABLE `maadb_project`.`tweet` (
			`id_tweet` INT NOT NULL AUTO_INCREMENT,
			`sentiment` VARCHAR(20) NOT NULL,
			`doc_number` INT NOT NULL,
			PRIMARY KEY (`id_tweet`),
			CONSTRAINT `sentiment_tweet`
			FOREIGN KEY (`sentiment`) REFERENCES `maadb_project`.`sentiment` (`nome`)
			ON DELETE RESTRICT ON UPDATE CASCADE);"""
exec_query(query)


#### Inserimento dati tabella tweet

In [15]:
def insert_tweet():
	insert = "('{}', '{}'),"
	for s in sentiments:
		query = """INSERT INTO `maadb_project`.`tweet` (`sentiment`, `doc_number`) VALUES"""
		for i in range(60000):
			query += insert.format(s, i)
		exec_query(query[:-1] + ";")
		print(f"Completed {s}")
insert_tweet()

Completed anger
Completed anticipation
Completed disgust
Completed fear
Completed hate
Completed hope
Completed joy
Completed like
Completed love
Completed sadness
Completed surprise
Completed trust
Completed pos
Completed neg


#### Creazione tabella wordsintweet

In [18]:
#exec_query("drop table maadb_project.wordsintweet")
query = """CREATE TABLE `maadb_project`.`wordsintweet` (
			`id_word` INT NOT NULL,
			`id_tweet` INT NOT NULL,
			`frequence` INT NOT NULL,
			`pos_tag` VARCHAR(4) NULL,
			PRIMARY KEY (`id_word`, `id_tweet`),
			CONSTRAINT `tweet_id`
				FOREIGN KEY (`id_tweet`) REFERENCES `maadb_project`.`tweet` (`id_tweet`)
				ON DELETE CASCADE ON UPDATE CASCADE,
			CONSTRAINT `word_id`
				FOREIGN KEY (`id_word`) REFERENCES `maadb_project`.`word` (`id_word`)
				ON DELETE CASCADE ON UPDATE CASCADE);"""
exec_query(query)

#### Inserimento dati tabella wordsintweet

In [14]:
#45 min
def build_words_dict_from_table():
    query = f"SELECT * FROM word"
    data = get_data_from_db(query).fetchall()
    lemma_dict = dict((lemma, id) for (id, lemma) in data)
    return lemma_dict

def build_tweet_dict_from_table():
    query = "SELECT * FROM tweet"
    data = get_data_from_db(query).fetchall()
    token_dict = dict(((sentiment, doc_number), id) for (id, sentiment, doc_number) in data)
    return token_dict

def insert_word_in_tweet_on_db():
    word_dict = build_words_dict_from_table()
    tweet_dict = build_tweet_dict_from_table()
    for s in sentiments:
        path = os.path.join(PREP_PATH, f"preprocessed_tweets_{s}.json")
        if os.path.exists(path):
            data = retrieve_data_from_json(path)
            query = build_query_string(data, word_dict, tweet_dict, s)
            exec_query(query)
            print(f"Inserted tweet of emotion {s}")

def build_query_string(data, word_dict, tweet_dict, sentiment):
    query = "INSERT INTO wordsintweet (id_word, id_tweet, frequence, pos_tag) VALUES"
    insert = '\n("{}", "{}", "{}", "{}"),'
    for tweet in data:
        inserted = []
        doc_number = tweet['doc_number']
        tweet_id = tweet_dict[(sentiment, doc_number)]
        for tweet_word in tweet['words']:
            lemma = tweet_word['lemma']
            if lemma not in inserted:
                word_id = word_dict[lemma]
                query += insert.format(word_id, tweet_id, tweet_word['freq'], tweet_word['pos_tag'])
                inserted.append(lemma)
    return query[:-1] + ";"

insert_word_in_tweet_on_db()

Inserted tweet of emotion anger
Inserted tweet of emotion anticipation
Inserted tweet of emotion disgust
Inserted tweet of emotion fear
Inserted tweet of emotion joy
Inserted tweet of emotion sadness
Inserted tweet of emotion surprise
Inserted tweet of emotion trust


#### Creazione tabella tokensintweet

In [ ]:
#exec_query("drop table maadb_project.tokenintweet")
query = """CREATE TABLE `maadb_project`.`tokensintweet` (
			`id_token` INT NOT NULL,
			`id_tweet` INT NOT NULL,
			`frequence` INT NOT NULL,
			PRIMARY KEY (`id_token`, `id_tweet`),
			CONSTRAINT `tweet_token_id`
				FOREIGN KEY (`id_tweet`) REFERENCES `maadb_project`.`tweet` (`id_tweet`)
				ON DELETE CASCADE ON UPDATE CASCADE,
			CONSTRAINT `token_id`
				FOREIGN KEY (`id_token`) REFERENCES `maadb_project`.`token` (`id_token`)
				ON DELETE CASCADE ON UPDATE CASCADE);"""
exec_query(query)

#### Inserimento dati in tabella tokensintweet

#### Creazione tabella wordsinlexres

In [16]:
#exec_query("drop table maadb_project.wordsinlexres")
query = """CREATE TABLE `maadb_project`.`wordsinlexres` (
			`id_word` INT NOT NULL,
			`id_lexres` INT NOT NULL,
			PRIMARY KEY (`id_word`, `id_lexres`),
			CONSTRAINT `id_word_lexres`
				FOREIGN KEY (`id_word`) REFERENCES `maadb_project`.`word` (`id_word`)
				ON DELETE CASCADE ON UPDATE CASCADE,
			CONSTRAINT `id_lexres`
				FOREIGN KEY (`id_lexres`) REFERENCES `maadb_project`.`lexical_resource` (`id_lex_res`)
				ON DELETE CASCADE ON UPDATE CASCADE);"""
exec_query(query)

#### Inserimento dati in tabella wordsinlexres

In [20]:
def retrieve_words_from_file(path):
    lines = []
    with open(path, 'r', encoding='utf8') as f:
        lines = [x.rstrip('\n') for x in f.readlines()]
    return [line for line in lines if "_" not in line]

def build_lex_res_word_dict():
    lex_res_word_dict = {}
    for sentiment in sentiments:
        words_of_res = []
        for res in resources[sentiment]:
            file_path = os.path.join(PATH, f'{sentiment.capitalize()}/{res}_{sentiment}.txt')
            words_of_res = retrieve_words_from_file(file_path)
            lex_res_word_dict[(res, sentiment)] = set(words_of_res)
    return lex_res_word_dict

def build_lex_res_dict_from_table():
    query = "SELECT * FROM lexical_resource"
    data = get_data_from_db(query).fetchall()
    token_dict = dict(((resource, sentiment), id) for id, sentiment, resource, _ in data)
    return token_dict

def insert_words_in_lex_res_in_db():
    lex_res_word_dict = build_lex_res_word_dict()
    words_dict = build_words_dict_from_table()
    lex_res_dict = build_lex_res_dict_from_table()
    insert_words_query = "INSERT INTO wordsinlexres (id_word, id_lexres) VALUES"
    insert = '\n("{}", "{}"),'
    for lex_res, words_of_res in lex_res_word_dict.items():
        for word in words_of_res:
            word_id = words_dict[word]
            lex_res_id = lex_res_dict[lex_res]
            insert_words_query += insert.format(word_id, lex_res_id)
    insert_words_query = insert_words_query[:-1] + ";"
    exec_query(insert_words_query)
    print("Words correctly inserted")

insert_words_in_lex_res_in_db()

Words correctly inserted
